# **Diseño y Desarrollo de un dashboard con Dash para desplegar los datos de las consultas realizadas a las APIs de EDAMAM** 

### *Por: Samuel Cortez / Maestría en Ciencia de Datos*

In [1]:
# Importar librerías para manipulación de los datos
import pandas as pd
import requests     
import logging
import numpy as np

# Importar librerías para el manejo de dash
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import plotly.graph_objects as go
import dash_bootstrap_components as dbc
import dash_table
import plotly.graph_objs as go
from dash import Input, Output, dcc, html

C:\Users\Samuel\AppData\Roaming\Python\Python37\site-packages\mpl_toolkits\mplot3d\__init__.py:1: MatplotlibDeprecationWarning: 
The deprecated function was deprecated in Matplotlib 3.4 and will be removed two minor releases later.
  from .axes3d import Axes3D
C:\Users\Samuel\.conda\envs\ML_ECNSCientific\lib\site-packages\ipykernel_launcher.py:9: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  if __name__ == '__main__':
C:\Users\Samuel\.conda\envs\ML_ECNSCientific\lib\site-packages\ipykernel_launcher.py:10: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  # Remove the CWD from sys.path while we load stuff.


### Solicitudes de recetas a la API de EDAMAM

In [2]:
# Se define clase Edamam_mcd
class Edamam_mcd:
    "API que regresa datos de tipo json"
    def __init__(self, 
                id_nutrition, 
                key_nutrition, 
                id_recipes,
                key_recipes,
                id_food,
                key_food):
        
        # Asignación de claves / ID de las API's
        self.id_nut = id_nutrition
        self.key_nut = key_nutrition
        self.id_rec = id_recipes
        self.key_rec = key_recipes
        self.id_food = id_food
        self.key_food = key_food
    
    def Nut_Analysis(self, query):
        self.Nut_ingredients = query
        # Se define url (Access Point) para realizar consulta a la API Nutrition Analysis
        url_req_nutr = 'https://api.edamam.com/api/nutrition-data'
        # Se definen los parametros a pasar por la función get para realizar consulta
        params_nutr= {
            'app_id':self.id_nut,
            'app_key':self.key_nut,
            'ingr':self.Nut_ingredients
            }
        # Envío de solicitud para realizar consulta de análisis nutricional
        Response = requests.get(url_req_nutr, params = params_nutr)

        # Imprimir el código del estatus
        print("Status Code (Nutrition Analysis API):", Response.status_code)
        
        if Response.status_code == 401:
            logging.error('{key} - Clave inválida (Nutrition Analysis API)'.format(key=self.key_nut))

        # Valor de retorno de tipo diccionario, Response tiene contenido JSON serializado
        self.r_Nut = Response.json()
        return self.r_Nut

    def Search_recipe(self, query):
        self.query_recipe = query

        # Se construye url con los parámetros para realizar consulta para busqueda de recetas de un platillo
        url_recipe = 'https://api.edamam.com/search?q={query}&app_id={id}&app_key={key}'.format(id=self.id_rec, key=self.key_rec,query=self.query_recipe)    

        Response = requests.get(url_recipe)
        #print('Response Recipe', Response)

        # Imprimir el código de estatus
        print("Status Code (Recipe Search API) :", Response.status_code)
        
        if Response.status_code == 401:
            logging.error('{key} - Clave inválida (Recipe Search API)'.format(key=self.key_rec))

        # Valor de retorno de tipo diccionario, Response tiene contenido JSON serializado 
        self.r_Recipe = Response.json()
        #print(self.r_Recipe)
        return self.r_Recipe

    def Search_food(self, query):
        self.query_food = query
        # Se define url (Access Point) para realizar consulta de análisis de comida
        url_food = 'https://api.edamam.com/api/food-database/parser?'
        # Se definen los parametros a pasar por la función get para realizar consulta
        params_food = {
            'app_id':self.id_food,
            'app_key':self.key_food,
            'ingr':self.query_food
            }
        # Se realiza una consulta dando la url
        Response = requests.get(url_food, params= params_food)
        
        # Imprimir el código de estatus
        print("Status Code (Food Database API):", Response.status_code)

        if Response.status_code == 401:
            logging.error('{key} - Clave inválida (Food Database API)'.format(key=self.key_rec))

        # Valor de retorno de tipo diccionario, Response tiene contenido JSON serializado
        self.r_Food = Response.json()
        return self.r_Food
    
    # Conjunto de funciones para generar dataframes con los datos principales de las consultas

    def Nutrient_Guide(self):
        # Manejo de excepciones
        try:
            # Se crean DataFrames de las variables totalNutrients, totalDaily y totalNutrientsKCal 
            self.df_Nutrition= pd.DataFrame(self.r_Nut.get('totalNutrients')).T.rename_axis(str(self.Nut_ingredients))
            self.df_totalDaily = pd.DataFrame(self.r_Nut.get('totalDaily')).T.rename_axis(str(self.Nut_ingredients))
            self.df_total_Nut =  pd.DataFrame(Response_Nut.get('totalNutrientsKCal')).T.rename_axis(str(self.Nut_ingredients))
            # Se obtiene las calorías del alimento
            self.Nutrient_Cal = self.r_Nut.get('calories')
            # Se obtiene el peso total del alimento
            self.totalWeight = self.r_Nut.get('totalWeight')
            
        except Exception as e:
            print(e)
        
    def ingredients_table(self):
        # Manejo de excepciones
        try:
            # Comprensión de listas: se crean nuevas listas con las etiquetas de la receta y con los ingredientes
            list_label = [element_hits.get('recipe').get('label') for element_hits in self.r_Recipe.get('hits')]
            #list_label = [element_hits.get('recipe').get('label') for element_hits in self.r_Recipe.get('hits')]
            list_ingredientes = [element_ingre.get('recipe').get('ingredients') for element_ingre in self.r_Recipe.get('hits')]
            # Se construye DataFrame 
            df_ = pd.DataFrame(list_ingredientes, index=list_label).stack().apply(pd.Series).drop(columns=['foodId'])
            
            # Se consutryen los DataFrames para desplegar en gráficos en el Dashboard
            # Se crean listas vacías
            food_list = []
            nutrient_list = []
            nutrient_label_list = []
            totalNut_list = []
            summary_r_list = []
            i = 0
            
            for element in self.r_Recipe['hits']:
                i+=1
                food_list.append(element.get('recipe').get('label'))
                nutrient_list.append([element.get('recipe').get('totalDaily').get(nutr_element).get('quantity') for nutr_element in element.get('recipe').get('totalDaily')])#[:6])
                totalNut_list.append([element.get('recipe').get('totalNutrients').get(nutr_element).get('quantity') 
                                      for nutr_element in element.get('recipe').get('totalNutrients') 
                                      if nutr_element != 'SUGAR.added'])
                # Calories
                summary_r_list.append([element.get('recipe').get('calories'), element.get('recipe').get('totalWeight'), element.get('recipe').get('totalTime')])
             
            # Indicadores Principales de la receta
            self.summary_r = pd.DataFrame(summary_r_list, columns = ['calories','totalWeight', 'totalTime'], index = food_list)
             
            nutrient_label_list = [element.get('recipe').get('totalDaily').get(nutr_element).get('label') for nutr_element in self.r_Recipe['hits'][0].get('recipe').get('totalDaily')]
            
            # Total Nutrient
            dict_totalNut = self.r_Recipe.get('hits')[0].get('recipe').get('totalNutrients')
            totalNut_label_list = [dict_totalNut.get(element).get('label') for element in dict_totalNut]
            totalNut_label_unit = [dict_totalNut.get(element).get('unit') for element in dict_totalNut]
            df_totalNut = pd.DataFrame(totalNut_list, columns = totalNut_label_list, index = food_list).fillna(0).T.reset_index(drop = False).rename(columns = {'index':'label'})
            self.df_totalNutrient = df_totalNut
            
            self.df_Recipe = df_.rename_axis(["Types", "Items"], axis = "rows").rename(columns = {"foodCategory": "food category", "text":"ingredient name"})
            # Tratamiento de datos para generar gráficos
            self.ingredients = pd.DataFrame([element[0] for element in self.df_Recipe.index.values.tolist()]).loc[:,0].unique().tolist()
            self.df_food = pd.DataFrame(nutrient_list, columns = nutrient_label_list, index =food_list) #[:6]
            #self.df_food_nut = self.df_food[['Carbs', 'Protein', 'Fat']].reset_index(drop = False).rename(columns={'index':'Food'})
            self.df_food_nut = self.df_food.reset_index(drop = False).rename(columns={'index':'Food'})
            self.df_food_scatter = self.df_food.reset_index(drop = False).rename(columns ={'index':'Food'})
            # Lista de comprensión imagenes
            self.image_recipe = pd.DataFrame([image.get('recipe').get('image') for image in self.r_Recipe.get('hits')], columns = ['image'], index = food_list)
            
        except Exception as e:
            print(e)

    def food_table(self):
        # Se generar listas vacias
        list_foods = []
        list_nutrients = []
        rename_nutrients = ['ENERGY (kcal)', 'PROTEIN (g)', 'FAT (g)', 'CARBS (g)', 'FIBER (g)']
        # Manejo de excepciones
        try:
            # Comprensión de listas: se crean nuevas listas con las etiquetas de los alimentos y con los nutrientes  
            list_foods = [element.get('food').get('label') for  element in self.r_Food['hints']]
            #print(list_foods)
            list_nutrients = [element.get('food').get('nutrients') for element in self.r_Food['hints']]
            # Se construye DataFrame
            self.df_food_table = pd.DataFrame(list_nutrients, index = list_foods).round(2).fillna('Unknown')
            self.df_food_table.columns = rename_nutrients
            
        except Exception as e:
            print(e) 
    
    def write_files(self):
        #  Exportar archivos .CSV/.xlsx
        path_Nutrition = '{Nut}_Nutritional_Analysis.xlsx'.format(Nut=self.Nut_ingredients)
        path_Recipe = '{Recipe}_Recipe.csv'.format(Recipe = self.query_recipe)
        path_food = '{food}_food.csv'.format(food = self.query_food)

        # Se exporta a excel el análisis nutricional 
        with pd.ExcelWriter(path_Nutrition) as writer:
            self.df_Nutrition.to_excel(writer, sheet_name= 'Nutritional_Analysis')
            self.df_totalDaily.to_excel(writer, sheet_name='Total_Daily')
            self.df_total_Nut.to_excel(writer, sheet_name='totalNutrientsKCal')
        # Se exporta en formato CSV los ingredientes de las recetas
        self.df_Recipe.to_csv(path_Recipe)
        # Se exporta en CSV el dataframe con el análisis de los alimentos
        self.df_food_table.to_csv(path_food)
        

# **Diseño de Dashboard con Dash**

In [ ]:
# Claves / ID's de las aplicaciones asignadas
nutrition_appid= '5f1e7888'
nutrition_appkey= '5576413f5acb38f5259a58dad888d1b9'
recipes_appid = 'dcda2d5e'
recipes_appkey= '87ae2d34cdde47026fc341c8fbd7957f'
food_appid= '6b69c52c'
food_appkey= 'd5a448b534a0c358dfb47fa7a6548724'

# Se crea el objeto EDAMAM_consulta
EDAMAM_consulta = Edamam_mcd(id_nutrition=nutrition_appid, 
                            key_nutrition= nutrition_appkey,
                            id_recipes=recipes_appid,
                            key_recipes=recipes_appkey,
                            id_food=food_appid,
                            key_food=food_appkey)

# Ejercución de los métodos de cada una de las API's para realizar consultas
Response_Recipe = EDAMAM_consulta.Search_recipe(query= 'salad')

# Ejecución de los métodos para obtener los dataframe con los resultados para cada una de las API's
EDAMAM_consulta.ingredients_table()


### Diseño Dash 2021/11/14

In [151]:

"""
Código para generar un Dashboard sobre información nutricional realizando consultas a las API's de EDAMAM

"""
# # Creación de figuras, y layouts en plotly - Dash

app = dash.Dash(
    __name__, meta_tags=[{"name":"viewport", "content":"width = device-width"}],
    external_stylesheets=[dbc.themes.BOOTSTRAP])

app.title = "Análisis Nutricional"

colors = {
    'background': '#C6D500',
    'text':'#111111',
}

fig  = go.Figure()
fig.update_layout(
    plot_bgcolor = colors['background'],
    paper_bgcolor = colors['background'],
    font_color = colors['text']
)

ingredients = [{'label': 'Zuni-Inspired Grilled Chicken Salad','value': 'Zuni-Inspired Grilled Chicken Salad'},
               {'label': 'Steak & Chips Salad', 'value': 'Steak & Chips Salad'},
               {'label': 'Shrimp Salad', 'value': 'Shrimp Salad'},
               {'label': 'Strawberry Hazelnut Salad', 'value': 'Strawberry Hazelnut Salad'},
               {'label': 'Grilled Tofu Salad With Miso Dressing','value': 'Grilled Tofu Salad With Miso Dressing'},
               {'label': 'Chicken Salad-Stuffed Tomatoes','value': 'Chicken Salad-Stuffed Tomatoes'},
               {'label': 'Buffalo Chicken Salad', 'value': 'Buffalo Chicken Salad'},
               {'label': 'Buffalo Chicken Salad recipes','value': 'Buffalo Chicken Salad recipes'},
               {'label': 'Washing up free salad', 'value': 'Washing up free salad'},
               {'label': 'Quinoa Salad', 'value': 'Quinoa Salad'}]


tab_selected_style = {
    "background": "gray",
    'text-transform': 'uppercase',
    'color' : '#BCBD22',
    'font-size': '15px',
    'font-weight': 'bold',
    'align-items': 'center',
    'justify-content': 'center',
    'border-radius': '8px',
    'padding':'6px',
    'border-style':'solid',
    'border-color':'gray',
    'background-color':'#384160',

}

tab_style = {
    'color': 'rgb(136,136,136)',
    'font-size': '15px',
    'font-weight': 'bold',
    'align-items': 'center',
    'justify-content': 'center',
    'border-radius': '8px',
    'padding':'6px',
    'border-style':'solid',
    'border-color':'gray',
}

server = app.server

app.layout= html.Div(
    children=[
        dbc.Container([
        dcc.Store(id="store"),
        dcc.Store(id ="store2"),
        dbc.Navbar(
            children=[
                html.A(
                    dbc.Row(
                        [
                            dbc.Col(html.Img(src ="https://images.plot.ly/logo/new-branding/plotly-logomark.png", height = "40px")),
                            dbc.Col([
                              dbc.NavbarBrand("Análisis Nutricional", class_name="ml_2",style={'font-size':38, 
                                                                                               'font-weight':'bold',
                                                                                               'textAlign':'center',
                                                                                               'color':'#384160'})], 
                            width="auto",align="center"),
                        ], 
                        align = "center",
                    ),
                ),
            ],
            color = "#c6d500",
            #align = "center",
            sticky = "center",
            ),

        html.H5(
            children = ["Analiza la información nutricional de recetas realizando consultas a la API Food Recipe de ",
                        html.A('EDAMAM', href = 'https://developer.edamam.com/edamam-docs-nutrition-api',target = '_blank')],
            style = {'font-weight':'bold','color':'#384160','textAlign':'left', 'fontsize':'56px'}
        ),
        html.Hr(style={'color':'black'}),
        html.H2('Búsqueda de alimento:', style = {'font-weight':'bold', 'color':'#384160'}),
        html.Hr(style={'color':'black'}),
        dcc.Input(
            placeholder = 'Enter a value...',
            id = "string_contr",
            type = 'text',
            value = 'Salad',
            style = {'display':'inline-block', 'font-weight':'bold', 'width':300,'height':40, 'font-size':25}
            ),
        html.Hr(style={'color':'black'}),
        dbc.Button(
            "Search",
            color = "primary",
            id = "button",
            className = "mb-3",
        ),
        dbc.Tabs(id="tabs",active_tab="ingredients",style=tab_selected_style,
            children = [
                dbc.Tab(label="Ingredientes", 
                        tab_id="ingredients",
                        label_style = {'color':'#29A3A8'},
                        children = [
                            dcc.Dropdown(
                                id ="Dropdown",
                                options = [{'label':k, 'value':k} for k in ingredients],
                                style = {'font-weight':'bold','font-size':20,'padding':2, 'color':'black'},
                                value = 'Strawberry Hazelnut Salad',
                                ),
                            ], 
                        ),

                dbc.Tab(label = "Valor Nutricional", 
                        tab_id="comparison",
                        label_style = {'color':'#29A3A8'},
                        ),
                dbc.Tab(label="Acerca de:", 
                        tab_id="Acerca",
                        label_style = {'color':'#29A3A8'},
                        ),
            ],
        ),
        html.Div(id="tab-content", className="p-4", style = {'color' :'black',
                                                             'border-color':'black',
                                                             'backgroundColor': 'white',#"rgb(136,136,136)",
                                                             'border-style':'solid',
                                                             'border-color':'gray',
                                                            }),#BAB0ACcolors['background']
    ], ),],style = {'color' :'black','backgroundColor':colors['background']})

@app.callback(
    Output("tab-content", "children"),
    [Input("tabs", "active_tab"), 
    Input("store", "data")])
def render_tab_content(active_tab, data):
    """
    This callback takes the 'active_tab' property as input, as well as the
    stored graphs, and renders the tab content depending on what the value of
    'active_tab' is.
    """
    if active_tab and data is not None:
        if active_tab == "ingredients":
            #print("Se activa el envío de data[bar_1]")
            return (html.Div([dbc.Row([html.H3(str(data["label_food"]))]),
                             dbc.Row([dbc.Col(html.Img(src = data["image"]), style = {'textAlign':'left'},), #width=4),
                                      dbc.Col(children = data["card_cont1"] , style = {'textAlign':'center','padding':15}),#, width= 3),
                                      dbc.Col(children = data["card_cont2"], style = {'textAlign':'center','padding': 15}),#, width= 3),
                                      #dbc.Col( style = {'textAlign':'center','padding': 0}),
                                      ],justify="left"),],
                            style = {'padding':10, 'flex':1, 'color':'#29A3A8', 'fontWeight':'bold'}), 
                    html.Div([html.H3(style={'color':"#29A3A8",'padding':25},children='Análisis por ingredientes'),
                              dbc.Col(dcc.Graph(figure=data["bar_1"])), 
                              dbc.Col(dcc.Graph(figure=data["bar_3"]))]))

        elif active_tab == "Acerca":
            return html.Div([
                dbc.Row([
                    dbc.Col(children=[
                        html.H3('Tabla - Guía de Nutrición'),
                        html.Hr(style={'color':'black'}),
                        dash_table.DataTable(
                            style_data={'whiteSpace': 'normal','height': 'auto'},
                            fill_width = False,
                            id = 'table_nut',
                            columns = [{"name":i, "id":i} for i in pd.DataFrame(data["Nut_guid"]).columns],
                            style_header={
                                'backgroundColor': 'gray',
                                'fontWeight': 'bold',
                                'border':'1px solid black'
                                },
                            style_cell = {'padding':'10px'},
                            style_as_list_view = True,
                            style_data_conditional=[
                                {
                                    'if': {'row_index': 'odd'},
                                    'backgroundColor': 'rgb(220, 220, 220)',
                                    }
                                ],
                            data = pd.DataFrame(data["Nut_guid"]).to_dict('records'),
                            ),
                        ],width="auto", align="center"),
                    ],justify = "center"),
                ],) 
                
        elif active_tab == "comparison":
            return html.Div([
                html.H3(style={'color':"#29A3A8",'padding':25},children='Valor Nutricional entre Alimentos'),
                dcc.Graph(figure = data["comparison"]),
                dcc.Graph(figure = data["scatter_1"])
                ])
    return "No tab selected"
 
 
@app.callback(Output("store", "data"),
              [Input("Dropdown", "value"),
              Input("store2","data")])

def generate_graphs(value, data_dict):
    """
    This callback generates three simple graphs from random data.
    """
    #print('generate_graph:', value)
    
    if not data_dict:
        # generate empty graphs when app loads
        return {k: go.Figure(data=[]) if k not in 
                ["image","label_food","card_cont1","card_cont2","card_cont3"] else "" 
                for k in [
                    "bar_1",
                    "scatter_1", 
                    "comparison",
                    "bar_3", 
                    "image", 
                    "label_food", 
                    "card_cont1",
                    "card_cont2",
                    "card_cont3"]}
    else:
        
        # Se importa tabla guída de nutrientes
        data_Nut = pd.read_csv('Nutrient_Guide.csv', index_col = None, sep = ',', encoding='utf-8')

        df_food_nut = pd.DataFrame(data_dict['df_food_nut'])
        df_food_scatter = pd.DataFrame(data_dict['df_food_scatter'])
        food = pd.DataFrame(data_dict['food'])
        df_image = pd.DataFrame(data_dict['df_image'])
        df_summary = pd.DataFrame(data_dict['df_summ'])
        df_TNut = pd.DataFrame(data_dict['df_TNutr'])
        
        ################################################################################
        # Scatter Protein vs Energy
        #fig_scatter = px.scatter(df_food_scatter, x="Protein", y="Energy", color="Food")
        fig_scatter = px.scatter_matrix(df_food_scatter, 
                                        dimensions=['Carbs', 'Protein', 'Fat','Fiber'], 
                                        color = "Food",height= 800)
        # Manipular el color del gráfico
        fig_scatter.update_layout(
            legend_title = 'Alimentos',
            paper_bgcolor="LightSteelBlue",
            font = dict(size = 16),
            title = '<b>Relación nutricional entre alimentos</b>',
            #font_color = "white",
            #  {'plot_bgcolor':'rgba(0,0,0,0)',
            #  'paper_bgcolor':'rgba(0,0,0,0)'}
            )
        fig_scatter.update_traces(diagonal_visible=False)
        ################################################################################

        # Se grafican las caracteristicas por ingredientes
        food_sorted = food.loc[lambda df: df['Types'] == value, ['ingredient name','weight','food']].sort_values(by = 'weight',ascending = True)

        
        ################################################################################
        # Objeto bar_1 -> Gráfico de barras 
        fig_bar1 =  px.bar(food_sorted, x = 'weight', y = 'ingredient name', orientation='h', color = 'food')
        # Manipular el color del gráfico
        fig_bar1.update_layout(
             legend_title = "Ingredientes",
             paper_bgcolor="LightSteelBlue"
            #  {'plot_bgcolor':'rgba(0,0,0,0)',
            #  'paper_bgcolor':'rgba(0,0,0,0)'}
            )
        fig_bar1.update_xaxes(title='<b>Peso, g</b>', visible=True, showticklabels=False)
        fig_bar1.update_layout(uniformtext_minsize=8, 
                               uniformtext_mode='hide', 
                               title = '<b>Porción por ingrediente</b>',
                               font = dict(size = 14))
        # Set the visibility ON
        fig_bar1.update_yaxes(title='<b>Ingredientes</b>', visible=True, showticklabels=False)
    
        ################################################################################
        # Objeto bar_2 -> Gráfico de barras 
        
        fig_bar2 = px.bar(df_food_nut, x = "Food", y = ['Carbs', 'Protein', 'Fat','Fiber'])#,width = 1250,height= 450)#, color = 'Food')
        fig_bar2.update_layout(
            legend_title = "Nutrientes",
            paper_bgcolor="LightSteelBlue")
        
        fig_bar2.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', font = dict(size = 14))
        # Set the visibility ON
        fig_bar2.update_xaxes(title='<b>Alimentos</b>', visible=True, showticklabels=False)
        fig_bar2.update_yaxes(title='<b>Valor, g</b>', visible=True, showticklabels=True)
        
        ################################################################################
        # Objeto bar_3 -> Gráfico de barras 
        fig_bar3 = px.bar(df_TNut,x= 'label', y = value, color ='label')
        fig_bar3.update_traces(texttemplate='%{text:.2s}', textposition='outside')
        fig_bar3.update_layout(title= "<b>Total Diario</b>" ,
                               uniformtext_minsize=8, 
                               uniformtext_mode='hide', 
                               legend_title = 'Nutrientes',
                               paper_bgcolor="LightSteelBlue",
                               font = dict(size = 14))
        # Set the visibility ON
        fig_bar3.update_xaxes(title='<b>Nutrientes</b>', visible=True, showticklabels=False)

        ################################################################################
        image_url = df_image.loc[value,"image"]
        
        # Resumen Receta calories
        calories = df_summary.loc[value,'calories']
        totalWeight = df_summary.loc[value,'totalWeight']
        totalTime = df_summary.loc[value, 'totalTime']
        
        card_content_cal = dbc.Card([dbc.CardHeader("CALORÍAS, cal"), 
                                 dbc.CardBody(
                                     [
                                         html.H5(str(round(calories,2)), className="card-title"),
                                         ]),
                                 ], color ="primary", outline = True, )#style = {"width":"18rem"})
        
        card_content_weight = dbc.Card([dbc.CardHeader("PESO TOTAL, g"), 
                                 dbc.CardBody(
                                     [
                                         html.H5(str(round(totalWeight,2)), className="card-title"),
                                         ]),
                                 ],color ="primary", outline = True, )#style = {"width":"18rem"})
        
        card_content_Time = dbc.Card([dbc.CardHeader("TIEMPO TOTAL"), 
                                 dbc.CardBody(
                                     [
                                         html.H5(str(round(totalTime,2)), className="card-title"),
                                         ]),
                                 ],color ="primary", outline = True)
        

        graph_dicts = {"bar_1": fig_bar1, 
                       "scatter_1": fig_scatter, 
                       "comparison": fig_bar2, 
                       "bar_3": fig_bar3,
                       "image": image_url, 
                       "label_food": value,
                       "card_cont1": card_content_cal,
                       "card_cont2": card_content_weight,
                       "card_cont3": card_content_Time,
                       "Nut_guid":data_Nut.to_dict()
                       }
        
        #print(graph_dicts)
        # save figures in a dictionary for sending to the dcc.Store
        return graph_dicts


@app.callback(Output("store2", "data"),
               Output("Dropdown", "options"),
              [Input("button", "n_clicks"), 
              Input("string_contr", "value")]) 
def query_Edamam(n, value):
    
    ctx = dash.callback_context
    print(value, "Si funciona")
    
    if ctx.triggered:
        #time.sleep(3)
        # Ejercución de los métodos de cada una de las API's para realizar consultas
        EDAMAM_consulta.Search_recipe(query= value)

        # Ejecución de los métodos para obtener los dataframe con los resultados para cada una de las API's
        #EDAMAM_consulta.Nutrient_Guide()
        EDAMAM_consulta.ingredients_table()
        #EDAMAM_consulta.food_table()
        
        food = EDAMAM_consulta.df_Recipe.reset_index()
        list_ingredients = EDAMAM_consulta.ingredients
        df_food_nut = EDAMAM_consulta.df_food_nut
        df_food_scatter = EDAMAM_consulta.df_food_scatter
        df_image = EDAMAM_consulta.image_recipe
        df_total_Nut = EDAMAM_consulta.df_totalNutrient
        
        # Recipe summary
        df_summary = EDAMAM_consulta.summary_r
        
        df_dicts ={"food":food.to_dict() ,
                   "df_food_nut":df_food_nut.to_dict(), 
                   "df_food_scatter":df_food_scatter.to_dict(),
                   "df_image":df_image.to_dict(),
                   "df_summ": df_summary.to_dict(),
                   "df_TNutr":df_total_Nut.to_dict()
                   }
       
        options_list = [{"label": element, "value":element} for element in list_ingredients]
        #df_json = json.dumps(df_dicts, indent = 3)
  
        return df_dicts, options_list
        
    else:
        print("NO EJECUTO API EDAMAM")
        #df_dicts ={"ingre":[], "df_food_nut":[], "df_food_scatter":[]}
        df_dicts = None
        ingred_non = [{'label': 'Zuni-Inspired Grilled Chicken Salad','value': 'Zuni-Inspired Grilled Chicken Salad'},
                      {'label': 'Steak & Chips Salad', 'value': 'Steak & Chips Salad'},
                      {'label': 'Shrimp Salad', 'value': 'Shrimp Salad'},
                      {'label': 'Strawberry Hazelnut Salad', 'value': 'Strawberry Hazelnut Salad'},
                      {'label': 'Grilled Tofu Salad With Miso Dressing','value': 'Grilled Tofu Salad With Miso Dressing'},
                      {'label': 'Chicken Salad-Stuffed Tomatoes','value': 'Chicken Salad-Stuffed Tomatoes'},
                      {'label': 'Buffalo Chicken Salad', 'value': 'Buffalo Chicken Salad'},
                      {'label': 'Buffalo Chicken Salad recipes','value': 'Buffalo Chicken Salad recipes'},
                      {'label': 'Washing up free salad', 'value': 'Washing up free salad'},
                      {'label': 'Quinoa Salad', 'value': 'Quinoa Salad'}]
        #list_ingredients = []
        return df_dicts, ingred_non
        
        #df_dicts = {"bar_1":[1,2,3,4]}

if __name__ == "__main__":
    #from waitress import serve
    #serve(app, host = "0.0.0.0", port = 8080)
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Nov/2021 18:18:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2021 18:18:50] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2021 18:18:50] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2021 18:18:50] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2021 18:18:50] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2021 18:18:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2021 18:18:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2021 18:18:51] "POST /_dash-update-component HTTP/1.1" 200 -


Salad Si funciona
NO EJECUTO API EDAMAM


127.0.0.1 - - [14/Nov/2021 18:18:51] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2021 18:18:51] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -


Salad Si funciona


127.0.0.1 - - [14/Nov/2021 18:18:54] "POST /_dash-update-component HTTP/1.1" 200 -


Status Code (Recipe Search API) : 200


127.0.0.1 - - [14/Nov/2021 18:18:55] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2021 18:18:55] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2021 18:18:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2021 18:19:16] "POST /_dash-update-component HTTP/1.1" 200 -
